In [3]:
import pandas as pd
import sqlite3

from sklearn.model_selection import train_test_split

In [4]:
conn = sqlite3.connect('pitching.sqlite')

In [6]:
pitch = pd.read_sql("SELECT * FROM pitch_clean", conn)
pitch.head()

,index,game_id,at_bat_number,pitch_number,pitch_type,batter_stand,pitcher_throws,batter,pitcher,player_name,...,pitch_offset_6,pitch_offset_7,pitch_offset_8,pitch_offset_9,pitch_offset_10,pitch_offset_11,pitch_offset_12,pitch_offset_13,pitch_offset_14,pitch_offset_15
0,0,716393,78,5,FF,R,R,664761,622098,"Smith, Drew",...,None,None,None,None,None,None,None,None,None,None
1,1,716393,78,4,FF,R,R,664761,622098,"Smith, Drew",...,None,None,None,None,None,None,None,None,None,None
2,2,716393,78,3,SL,R,R,664761,622098,"Smith, Drew",...,None,None,None,None,None,None,None,None,None,None
3,3,716393,78,2,SL,R,R,664761,622098,"Smith, Drew",...,None,None,None,None,None,None,None,None,None,None
4,4,716393,78,1,FF,R,R,664761,622098,"Smith, Drew",...,None,None,None,None,None,None,None,None,None,None


In [7]:
pitch.columns

Index(['index', 'game_id', 'at_bat_number', 'pitch_number', 'pitch_type',
       'batter_stand', 'pitcher_throws', 'batter', 'pitcher', 'player_name',
       'zone', 'balls', 'strikes', 'description',
       'estimated_ba_using_speedangle', 'estimated_woba_using_speedangle',
       'launch_speed', 'launch_angle', 'inning', 'pitch_group', 'count_type',
       'score_diff', 'key_mlbam', 'batter_name', 'on_1b_ind', 'on_2b_ind',
       'on_3b_ind', 'pitch_num', 'pitch_1', 'pitch_10', 'pitch_11', 'pitch_12',
       'pitch_13', 'pitch_14', 'pitch_15', 'pitch_16', 'pitch_2', 'pitch_3',
       'pitch_4', 'pitch_5', 'pitch_6', 'pitch_7', 'pitch_8', 'pitch_9',
       'pitch_offset_1', 'pitch_offset_2', 'pitch_offset_3', 'pitch_offset_4',
       'pitch_offset_5', 'pitch_offset_6', 'pitch_offset_7', 'pitch_offset_8',
       'pitch_offset_9', 'pitch_offset_10', 'pitch_offset_11',
       'pitch_offset_12', 'pitch_offset_13', 'pitch_offset_14',
       'pitch_offset_15'],
      dtype='object')

In [8]:
# convert everything to float or bools for efficiency
pitch['batter_r'] = pitch['batter_stand'] == 'R'
pitch['pitcher_r'] = pitch['pitcher_throws'] == 'R'
pitch['is_fastball'] = pitch['pitch_group'] == 'FASTBALL'
pitch['is_behind'] = pitch['count_type'] == 'BEHIND'
pitch['is_ahead'] = pitch['count_type'] == 'AHEAD'
pitch['is_even'] = pitch['count_type'] == 'EVEN'

field_name = ''
offset_name = ''
for x in range(1, 6):
    field_name = f'pitch_offset_fb_{str(x)}'
    offset_name = f'pitch_offset_{str(x)}'
    pitch[field_name] = pitch[offset_name] == 'FASTBALL'

In [10]:
# Keeping features to predict on
# only keep ing 5 pitch offsets, avg. MLB at bat is 5 pitches
pitch_feats_fields = [ 
    'batter_r',
    'pitcher_r',
    'inning',
    'score_diff',
    'is_behind',
    'is_ahead',
    'is_even',
    'on_1b_ind',
    'on_2b_ind',
    'on_3b_ind',
    'pitch_offset_fb_1',
]

pitch_feats = pitch[pitch_feats_fields]
pitch_feats.head()

,batter_r,pitcher_r,inning,score_diff,is_behind,is_ahead,is_even,on_1b_ind,on_2b_ind,on_3b_ind,pitch_offset_fb_1
0,True,True,9,7,False,True,False,0,0,0,True
1,True,True,9,7,False,True,False,0,0,0,False
2,True,True,9,7,False,True,False,0,0,0,False
3,True,True,9,7,False,True,False,0,0,0,True
4,True,True,9,7,False,False,True,0,0,0,False


In [11]:
pitch_labels = pitch['is_fastball']
pitch_labels.head()

0     True
1     True
2    False
3    False
4     True
Name: is_fastball, dtype: bool

In [12]:
x_train, x_test, y_train, y_test = train_test_split(pitch_feats, pitch_labels, test_size = 0.4, random_state=42)

In [13]:
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size = 0.5, random_state=42)

In [14]:
# Aiming for train dataset of 60% total recurds, test data set of 20%, validation set of 20%
for dataset in [y_train, y_val, y_test]:
    print(round(len(dataset)/len(pitch_labels), 2)*100)

60.0
20.0
20.0


In [15]:
x_train.to_sql('train_features', conn, index=False, if_exists='replace')
x_test.to_sql('test_features', conn, index=False, if_exists='replace')
x_val.to_sql('val_features', conn, index=False, if_exists='replace')

y_train.to_sql('train_labels', conn, index=False, if_exists='replace')
y_test.to_sql('test_labels', conn, index=False, if_exists='replace')
y_val.to_sql('val_labels', conn, index=False, if_exists='replace')

148608

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=766ac258-4b25-4aeb-9ca3-1c0ba0991c26' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>